# NAS with image classifier on Cifar10


Firstly install the dependencies by executing the following command

!pip install autokeras

Import the required libraries

In [3]:
import numpy as np
import tensorflow as tf
#from tensorflow.keras.datasets import mnist
import tensorflow_datasets as tfds
import autokeras as ak

Here we loads the CIFAR-10 dataset using the TensorFlow Keras API. 

In [4]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 [==============================] - 30s 0us/step


Now trains an image classification model using the AutoKeras library and evaluates it on the test set.
    First, an instance of the ImageClassifier class from AutoKeras is initialized with overwrite=True to allow the model to be retrained if necessary, and max_trials=1 to limit the number of different models that AutoKeras will train.

In [5]:
# Initialize the image classifier.
clf = ak.ImageClassifier(overwrite=True, max_trials=1)
# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)


# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)


# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))


Trial 1 Complete [00h 01m 33s]
val_loss: 0.8864099383354187

Best val_loss So Far: 0.8864099383354187
Total elapsed time: 00h 01m 33s
Epoch 1/10
1563/1563 [==============================] - 9s 5ms/step - loss: 1.3394 - accuracy: 0.5269
Epoch 2/10
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0583 - accuracy: 0.6318
Epoch 3/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9664 - accuracy: 0.6637
Epoch 4/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9066 - accuracy: 0.6845
Epoch 5/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8691 - accuracy: 0.6957
Epoch 6/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8351 - accuracy: 0.7067
Epoch 7/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8041 - accuracy: 0.7192
Epoch 8/10
1563/1563 [==============================] - 8s 5ms/step - loss: 0.7898 - accuracy: 0.7261
Epoch 9/10
1563/1563 [============================

313/313 [==============================] - 1s 2ms/step
[['3']
 ['8']
 ['0']
 ...
 ['3']
 ['4']
 ['7']]
313/313 [==============================] - 1s 3ms/step - loss: 0.8614 - accuracy: 0.7096
[0.8613971471786499, 0.7095999717712402]


# Customized Search Space
Here use AutoKeras (ak) to create an Image Classification model with a customized search space. setting the block_type argument to "resnet" specifies that only ResNet architectures will be considered during the architecture search process.After that A ClassificationHead is added to the output node to perform the classification task.
AutoKeras will search for the best model architecture and hyperparameters based on the training data provided.


In [6]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)
clf.fit(x_train, y_train, epochs=10)


Trial 1 Complete [01h 00m 31s]
val_loss: 2.3249871730804443

Best val_loss So Far: 2.3249871730804443
Total elapsed time: 01h 00m 31s
Epoch 1/10
1563/1563 [==============================] - 419s 251ms/step - loss: 6.7424 - accuracy: 0.0993
Epoch 2/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0994
Epoch 3/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0994
Epoch 4/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0993
Epoch 5/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0993
Epoch 6/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0993
Epoch 7/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0993
Epoch 8/10
1563/1563 [==============================] - 389s 249ms/step - loss: 2.3148 - accuracy: 0.0993
Epoch 9/10
1563/15

Here by using AutoKeras to create an Image Classification model, but with a slightly different custom search space like The ResNetBlock layer is added to the output node. In this case, the version argument is set to "v2", which specifies the version of the ResNet architecture to use.
An AutoModel is created using the input and output nodes. The overwrite argument is set to True, which means that if there is an existing model with the same name, it will be overwritten. The max_trials argument is set to 2, which means that AutoKeras will run two trials to find the best model architecture.

In [6]:
input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=2
)
clf.fit(x_train, y_train, epochs=10)

Trial 2 Complete [00h 11m 02s]
val_loss: 1.5490269660949707

Best val_loss So Far: 1.5490269660949707
Total elapsed time: 00h 23m 45s
Epoch 1/10
1563/1563 [==============================] - 71s 39ms/step - loss: 2.1652 - accuracy: 0.2246
Epoch 2/10
1563/1563 [==============================] - 63s 40ms/step - loss: 1.8358 - accuracy: 0.3375
Epoch 3/10
1563/1563 [==============================] - 62s 39ms/step - loss: 1.6673 - accuracy: 0.3949
Epoch 4/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.5398 - accuracy: 0.4416
Epoch 5/10
1563/1563 [==============================] - 63s 40ms/step - loss: 1.3922 - accuracy: 0.5009
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.2110 - accuracy: 0.5751
Epoch 7/10
1563/1563 [==============================] - 62s 39ms/step - loss: 1.0046 - accuracy: 0.6611
Epoch 8/10
1563/1563 [==============================] - 62s 39ms/step - loss: 0.7691 - accuracy: 0.7625
Epoch 9/10
1563/1563 [============

In [7]:
import tensorflow as tf
import tensorflow_datasets as tfds
import autokeras as ak

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert labels to categorical one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

# Define AutoKeras model
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, outputs=output_node, overwrite=True, max_trials=1
)

# Train AutoKeras model on CIFAR-10
clf.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

# Evaluate AutoKeras model on test data
score = clf.evaluate(x_test, y_test)
print('Test accuracy:', score[1])


Trial 1 Complete [00h 13m 58s]
val_loss: 1.2210843563079834

Best val_loss So Far: 1.2210843563079834
Total elapsed time: 00h 13m 58s
Epoch 1/10
1563/1563 [==============================] - 110s 51ms/step - loss: 2.1473 - accuracy: 0.3168 - val_loss: 9.3872 - val_accuracy: 0.3801
Epoch 2/10
1563/1563 [==============================] - 78s 50ms/step - loss: 2.1181 - accuracy: 0.3396 - val_loss: 272.2827 - val_accuracy: 0.1732
Epoch 3/10
1563/1563 [==============================] - 77s 50ms/step - loss: 2.1170 - accuracy: 0.3232 - val_loss: 2.1868 - val_accuracy: 0.2164
Epoch 4/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.8794 - accuracy: 0.3572 - val_loss: 1.6193 - val_accuracy: 0.4526
Epoch 5/10
1563/1563 [==============================] - 77s 50ms/step - loss: 1.6035 - accuracy: 0.4465 - val_loss: 1.4571 - val_accuracy: 0.4716
Epoch 6/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.4598 - accuracy: 0.5023 - val_loss: 1.3909 - val_accu

313/313 [==============================] - 6s 14ms/step - loss: 1.1586 - accuracy: 0.6064
Test accuracy: 0.6064000129699707


# Auto model
Here use AutoKeras to build an image classification model for the CIFAR-10 dataset
Define an AutoModel using the AutoKeras API. The model consists of an image input, an image block, a flatten layer, a dense block, and a classification head.

The max_trials parameter of the AutoModel constructor is set to 2, which means that AutoKeras will try two different model architectures before returning the best one. 

In [8]:
import autokeras as ak
import torch
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Load the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert labels to categorical format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the AutoModel
input_node = ak.ImageInput()
output_node = ak.ImageBlock()(input_node)
output_node = ak.Flatten()(output_node)
output_node = ak.DenseBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

# Initialize the AutoModel
clf = ak.AutoModel(
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=2
)

# Fit the AutoModel to the CIFAR-10 dataset
clf.fit(x_train, y_train, epochs=10)

# Evaluate the AutoModel on the test data
accuracy = clf.evaluate(x_test, y_test)[1]
print('Accuracy: {:.2f}%'.format(accuracy * 100))


Trial 2 Complete [00h 12m 02s]
val_loss: 0.9828464984893799

Best val_loss So Far: 0.9828464984893799
Total elapsed time: 00h 22m 55s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1563/1563 [==============================] - 86s 54ms/step - loss: 1.7546 - accuracy: 0.3557
Epoch 2/10
1563/1563 [==============================] - 77s 49ms/step - loss: 1.3282 - accuracy: 0.5233
Epoch 3/10
1563/1563 [==============================] - 77s 49ms/step - loss: 1.1634 - accuracy: 0.5883
Epoch 4/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.0757 - accuracy: 0.6203
Epoch 5/10
1563/1563 [==============================] - 78s 50ms/step - loss: 1.0078 - accuracy: 0.6438
Epoch 6/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.9644 - accuracy: 0.6616
Epoch 7/10
1563/1563 [==============================] - 77s 50ms/step - loss: 0.9225 - accuracy: 0.6756
Epoch 8/10
1563/1563 [==============================] - 77s 49ms/step - loss: 0.8880 - accuracy: 0.6

INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


313/313 [==============================] - 3s 9ms/step - loss: 1.0022 - accuracy: 0.6558
Accuracy: 65.58%
